# Unofficial ACI Guide

## Python 3 Authentication Example

This is a simple Python example demonstrating authentication.  
No Cisco ACI-related toolkits, SDKs, or bindings were harmed in the creation of this example.

In [5]:
import requests
import json

Enter your controller hostname/IP and user credentials below.  
You probably don't want to do this while someone watches over your shoulder.  
Some examples for reading user info from a config file and prompted are also included.

In [6]:
# Variables to construct our URLs.
# Configure username, password, and controller.
# You can use inline, a separate config file, or prompted on the command-line.
# Just uncomment which one you wish to use.

# Inline:
#controller = "wabac-machine.mr-peabody.com"
#username = "sherman"
#password = "and-away-we-go!"

# From a config.py file:
from config import controller, username, password 

# Prompted for user/pass on the command line:
#controller = input('Controller hostname or IP address: ')
#username = input('Username: ')
#password = input('Password: ')

base_url = "https://" + str(controller) + "/api/"
auth_bit = "aaaLogin.json"
auth_url = base_url + auth_bit

In [7]:
# JSON data constructed from user/pass above.

auth_data = {
  "aaaUser":{
    "attributes":{
      "name":username,
      "pwd":password
    }
  }
}

### WARNING! PASSWORDS PRINTED BELOW! 

In [9]:
# Delete or comment out the print statment below to avoid this.
# Or, don't run this cell. It's just informational to demostrate 
# what the combined string looks like.

# Here's the assembled auth URL and the data we need to POST to it. 
#print("Auth URL: " + auth_url + "\n" + "Auth Data: " + str(auth_data) + "\n")

## Construct the Request

In [13]:
# "requests.post" sends an HTTP POST to auth_url shown above.
# "json=auth_data" encodes the payload auth_data as json
# "verify=False" means we will not try to validate the self-signed SSL ccertificate our controller uses

r = requests.post(auth_url, json=auth_data, verify=False)

# This prints the HTTP response, such as 200, 403, 404, 500, etc.
# You could use this for error trapping.
print("Server responsded with " + str(r.status_code))

Server responsded with 200


/anaconda2/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [14]:
# We're taking the response, r, and running the json() method to decode JSON 
# into native Python data structures, like list and dictionaries. We'll store
# that into a variable called "r_json" to work with.
r_json = r.json()

In [15]:
# The output is a single dictionary with two keys ("imdata" and "totalCount"), and a whole lotta stuff.
r_json

{'totalCount': '1',
 'imdata': [{'aaaLogin': {'attributes': {'token': 'dIwGAAAAAAAAAAAAAAAAAJA8mHDjahqU4dPrVHCS7IFIb9ELpOrH+09z+waSfk8RrMxFIQx2x0bUbN1S5I8N911PyvdzjbEQpnP9FQGsMz5U7Nv7GxnTmSHsYYHH5rD4BY+q0I7NpyPI0ERnFOU3pqT8FJ2nUJFskhAtOekyum+f/E1Eh/sEHMO0VbAoTrQxCLZEqwaMPUdefkXFqgsf0xE1+n6bYqadq61lv3UC3hjC0tHV6hBCpwAxWwqInf0E',
     'siteFingerprint': '2hlEKPs8VI2HTh7E',
     'refreshTimeoutSeconds': '3600',
     'maximumLifetimeSeconds': '86400',
     'guiIdleTimeoutSeconds': '3600',
     'restTimeoutSeconds': '90',
     'creationTime': '1551387608',
     'firstLoginTime': '1551387608',
     'userName': 'admin',
     'remoteUser': 'false',
     'unixUserId': '15374',
     'sessionId': '16fs3up8StaGozM+mQxEMA==',
     'lastName': '',
     'firstName': '',
     'changePassword': 'no',
     'version': '4.0(3d)',
     'buildTime': 'Thu Feb 07 16:20:39 PST 2019',
     'node': 'topology/pod-1/node-1'},
    'children': [{'aaaUserDomain': {'attributes': {'name': 'common',
        'rolesR': 'v

In [16]:
# Pretty Print, pretty please?
print(json.dumps(r_json, indent=4, sort_keys=True))

# The output is easier to visualize now. We see we have two dictionaries; one is called 
# "totalCount" and can be used to cross-check the number of objects returned. 
# The other is "imdata", whose value is a list of objects (in this case, a single object 
# called aaaLogin, matching that totalCount above). The aaaLogin object includes attributes
# and child objcets with their attributes.

{
    "imdata": [
        {
            "aaaLogin": {
                "attributes": {
                    "buildTime": "Thu Feb 07 16:20:39 PST 2019",
                    "changePassword": "no",
                    "creationTime": "1551387608",
                    "firstLoginTime": "1551387608",
                    "firstName": "",
                    "guiIdleTimeoutSeconds": "3600",
                    "lastName": "",
                    "maximumLifetimeSeconds": "86400",
                    "node": "topology/pod-1/node-1",
                    "refreshTimeoutSeconds": "3600",
                    "remoteUser": "false",
                    "restTimeoutSeconds": "90",
                    "sessionId": "16fs3up8StaGozM+mQxEMA==",
                    "siteFingerprint": "2hlEKPs8VI2HTh7E",
                    "token": "dIwGAAAAAAAAAAAAAAAAAJA8mHDjahqU4dPrVHCS7IFIb9ELpOrH+09z+waSfk8RrMxFIQx2x0bUbN1S5I8N911PyvdzjbEQpnP9FQGsMz5U7Nv7GxnTmSHsYYHH5rD4BY+q0I7NpyPI0ERnFOU3pqT8FJ2nUJFskhAtOekyum+f/E1

## Tokens and Cookies

In [17]:
# We can extract the session token from the response body.
# We need to reference the dictionary key "imdata",
# whose value is a list with one element "aaaLogin", which happens to be a dictionary itself.
# The value of the aaaLogin dictionary key includes a set of atttributes (more dictionaries!)
# and child objects for MOAR dictionaries!
#
# You can traverse this mess by referencing the r_json object/variable, and then using
# each of the dictionary keys and list elements to get to the exact key/value pair we need.
#
# For example:
# r_json -> imdata (dict key) -> first object in list (list element "0") -> aaaLogin object (dict key) ->
#    attributes (dict key) -> token (dict key) -> Value of that token


token = r_json["imdata"][0]["aaaLogin"]["attributes"]["token"]
print(token)

IrwDAAAAAAAAAAAAAAAAAE0ZnEMtmq0mymgjdI/DyKiQvW7YE6XC4DuOeOnvwtomDTDs0+ClkMLS1kmfXZoS8EwvitIZhwZ/nZAMyGlEdnlmYSaIuu1wNx2re1nZ+5Uws45h7Sxu+YZI2vnoYcfdymDaWe+51SGGWkplKHcCEx0PbVnoZ+3c2BaY1HsFljowp/0e0L18ONoQTXmEacsBRYA7blH1yIti8H6wU+OL0VhuRZaaXws6dGtWIXAO4k+2


In [28]:
# We can also view the HTTP headers from the initial response.
# HTTP headers are captured as a single dictionary with each header key/value
# mapping to a dictionary key/value.
r.headers

{'Server': 'Cisco APIC', 'Date': 'Mon, 25 Feb 2019 19:26:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '4259', 'Connection': 'keep-alive', 'Set-Cookie': 'APIC-cookie=CGwFAAAAAAAAAAAAAAAAABTzTQQDkckm+FwiLTWlzJzj1UA7w+I4QN5irxC8Wj8XJOUK0LmFTAj4MnGJt9B0ZAbrBT3j0L1y9cOzlS71FdbWkwk2F3odOwl7EoyUUbuZaMcHyvEWY1NcbbH9+WK6Par3l/MNzsWPXmeOMbyGvZCZxRIskNcWVim/xLXwNcsrVSD5uFZTZ2dE9s7okcdT3MJiz5IWIUtyF/Q7fEmPtrVibzeam5Nm6F+hDfAoS0yL; path=/; HttpOnly; HttpOnly; Secure', 'Access-Control-Allow-Headers': 'Origin, X-Requested-With, Content-Type, Accept, devcookie, APIC-challenge', 'Access-Control-Allow-Methods': 'POST,GET,OPTIONS,DELETE', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'Cache-Control': 'no-cache="Set-Cookie, Set-Cookie2"', 'Client-Cert-Enabled': 'false', 'Access-Control-Allow-Origin': 'http://127.0.0.1:8000', 'Access-Control-Allow-Credentials': 'true'}

In [18]:
# The HTTP headers include the token as a cookie.
# We can extract the HTTP header directly, which is formatted like:
# Set-Cookie: APIC-cookie=[token]
# Show the cookie in the HTTP headers
cookie_header = r.headers['Set-Cookie']
cookie_header

'APIC-cookie=IrwDAAAAAAAAAAAAAAAAAE0ZnEMtmq0mymgjdI/DyKiQvW7YE6XC4DuOeOnvwtomDTDs0+ClkMLS1kmfXZoS8EwvitIZhwZ/nZAMyGlEdnlmYSaIuu1wNx2re1nZ+5Uws45h7Sxu+YZI2vnoYcfdymDaWe+51SGGWkplKHcCEx0PbVnoZ+3c2BaY1HsFljowp/0e0L18ONoQTXmEacsBRYA7blH1yIti8H6wU+OL0VhuRZaaXws6dGtWIXAO4k+2; path=/; HttpOnly; HttpOnly; Secure'

In [19]:
# The requests module also can extract the cookie directly, by searching for the header key ("Set-Cookie"), 
# and reading its cookie value "APIC-cookie=[token]". This is a key=value pair, so this extracts the cookie value
# which happily matches the session token above. 

# Accessing the cookie directly
cookie = r.cookies['APIC-cookie']
cookie

'IrwDAAAAAAAAAAAAAAAAAE0ZnEMtmq0mymgjdI/DyKiQvW7YE6XC4DuOeOnvwtomDTDs0+ClkMLS1kmfXZoS8EwvitIZhwZ/nZAMyGlEdnlmYSaIuu1wNx2re1nZ+5Uws45h7Sxu+YZI2vnoYcfdymDaWe+51SGGWkplKHcCEx0PbVnoZ+3c2BaY1HsFljowp/0e0L18ONoQTXmEacsBRYA7blH1yIti8H6wU+OL0VhuRZaaXws6dGtWIXAO4k+2'

In [20]:
# In order to send the token with subsequent requests, we'll need to send the cookie back as an HTTP header.
# The requests module will asssemble a cookie from a dictionary "key:value" pair. 
# We could also pass multiple cookies as a dictionary with multiple key:value pairs, like so:
# cookies = {'Cookie1': some_value, 'Cookie2': some_other_value}
# Of course, multiple cookies are contained in a cookie jar, right? Even if there's only one cookie left!
cookie_jar = {'APIC-cookie': cookie}
cookie_jar

# We could also use that session token above, if we wanted to, since we know the APIC requires the cookie 
# to be in the format "APIC-cookie".
token_cookie_jar = {'APIC-cookie': token}
token_cookie_jar
print("Using cookie header: " + "\n" + str(cookie_jar) + "\n\n" + "Using token: " + "\n" + str(token_cookie_jar))

Using cookie header: 
{'APIC-cookie': 'IrwDAAAAAAAAAAAAAAAAAE0ZnEMtmq0mymgjdI/DyKiQvW7YE6XC4DuOeOnvwtomDTDs0+ClkMLS1kmfXZoS8EwvitIZhwZ/nZAMyGlEdnlmYSaIuu1wNx2re1nZ+5Uws45h7Sxu+YZI2vnoYcfdymDaWe+51SGGWkplKHcCEx0PbVnoZ+3c2BaY1HsFljowp/0e0L18ONoQTXmEacsBRYA7blH1yIti8H6wU+OL0VhuRZaaXws6dGtWIXAO4k+2'}

Using token: 
{'APIC-cookie': 'IrwDAAAAAAAAAAAAAAAAAE0ZnEMtmq0mymgjdI/DyKiQvW7YE6XC4DuOeOnvwtomDTDs0+ClkMLS1kmfXZoS8EwvitIZhwZ/nZAMyGlEdnlmYSaIuu1wNx2re1nZ+5Uws45h7Sxu+YZI2vnoYcfdymDaWe+51SGGWkplKHcCEx0PbVnoZ+3c2BaY1HsFljowp/0e0L18ONoQTXmEacsBRYA7blH1yIti8H6wU+OL0VhuRZaaXws6dGtWIXAO4k+2'}


In [21]:
# Now we can construct an authenticated GET using the token or cookie we've extracted and assembled. 
# How about getting all endpoints in the system?
ep_url = base_url + "node/class/fvCEp.json"
response = requests.get(ep_url, cookies=cookie_jar, verify=False)
out = response.json()
out

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'totalCount': '8',
 'imdata': [{'fvCEp': {'attributes': {'annotation': '',
     'childAction': '',
     'contName': '',
     'dn': 'uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8D:43',
     'encap': 'vlan-222',
     'extMngdBy': '',
     'id': '0',
     'idepdn': '',
     'ip': '192.168.50.12',
     'lcC': 'learned',
     'lcOwn': 'local',
     'mac': '00:DE:FB:79:8D:43',
     'mcastAddr': 'not-applicable',
     'modTs': '2019-02-15T11:58:25.443-06:00',
     'monPolDn': 'uni/tn-common/monepg-default',
     'name': '00:DE:FB:79:8D:43',
     'nameAlias': '',
     'status': '',
     'uid': '0',
     'uuid': '',
     'vmmSrc': ''}}},
  {'fvCEp': {'attributes': {'annotation': '',
     'childAction': '',
     'contName': '',
     'dn': 'uni/tn-COAST/ctx-COAST_vrf/cep-00:DE:FB:79:8B:C3',
     'encap': 'vlan-50',
     'extMngdBy': '',
     'id': '0',
     'idepdn': '',
     'ip': '192.168.50.11',
     'lcC': 'learned',
     'lcOwn': 'local',
     'mac': '00:DE:FB:79:8B:C3',
     'mcastAddr': 'n

In [22]:
# Note the "totalCount" object, and how it matches the number fvCEp objects in the imdata list returned...
# Print the value of the totalCount dictionary key.
print("Total Count: " + out["totalCount"] + " objects.")

# The imdata key has a list value, so we'll count the number of list elements using len()
count_of_fvCEp = len(out['imdata'])
print("Total Count: " + str(count_of_fvCEp) + " fvCEp list elements.")

# If those numbers don't match, either the APIC has a problem or you do.

Total Count: 8 objects.
Total Count: 8 fvCEp list elements.


## Sessions

In [23]:
# If you don't want to parse and pass token on each request, 
# just use requests session object which will retain persistent data (like cookies) for you.
# http://docs.python-requests.org/en/master/user/advanced/

# Sessions will also keep the TCP connection open for subsequent requests. 

# WARNING! PASSWORDS PRINTED BELOW! 
# Delete or comment out the print statment below to avoid this.
# Or, don't run this cell. It's just informational.

# Here's the assembled auth URL and the data we need to POST to it. 
print("Auth URL: " + auth_url + "\n" + str(auth_data) + "\n")

# The GET request for endpoints:
print("Endpoints URL: " + ep_url)

Auth URL: https://10.18.188.101/api/aaaLogin.json
{'aaaUser': {'attributes': {'name': 'sherman', 'pwd': 'and-away-we-go!'}}}

Endpoints URL: https://10.18.188.101/api/node/class/fvCEp.json


In [24]:
# Create a session object, s, and then POST the auth data just like the earlier example. 
# We don't need to create a new request object since the session will persist that data in the session object.
s = requests.session()
s.post(auth_url, json=auth_data, verify=False)

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<Response [200]>

In [25]:
# Note the lack of cookie headers from the requests object above:
# response = requests.get(ep_url, cookies=cookie_jar, verify=False)
session_response = s.get(ep_url, verify=False)

/anaconda2/envs/aci/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [26]:
s_out = session_response.json()

In [27]:
print("Total Count: " + s_out["totalCount"] + " objects.")

count_of_fvCEp = len(s_out['imdata'])
print("Total Count: " + str(count_of_fvCEp) + " fvCEp list elements.")

Total Count: 8 objects.
Total Count: 8 fvCEp list elements.
